I - Analyses préliminaires : 

=== 1. Import des librairies ===

In [ ]:
import pandas as pd
import numpy as np

=== 2. Lecture du fichier ===

In [ ]:
data_path = "data/hourly_data.csv"
df = pd.read_csv(data_path)

=== 3. Aperçu général ===

In [ ]:
print("Nombre de lignes :", df.shape[0])
print("Nombre de colonnes :", df.shape[1])

display(df.head())

=== 4. Infos générales ===

In [ ]:
print("\n=== Informations générales sur le dataset ===")
df.info()

=== 5. Vérification des valeurs manquantes ===

In [ ]:
print("\n=== Pourcentage de valeurs manquantes par colonne ===")
missing = df.isna().mean() * 100
display(missing[missing > 0].sort_values(ascending=False))

--> 2 variables ayant des valeurs manquantes au taux de moins de 2% (taux faible)
--> 3 options : 
        - supprimer les lignes ayant des valeurs manquantes et suivre le processus
        - imputer les valeurs manquantes`: interpolation temporelle, préférable pour la modélisation 
        - voir l'importance de ces deux variables dans la modélisation et voir si c'est intéressant de les supprimer

=== 6. Localisation des valeurs manquantes et interprétations ===

In [ ]:
missing_rows = df[df['boundary_layer_height'].isna() | df['total_column_integrated_water_vapour'].isna()]
print("Nombre de lignes concernées :", len(missing_rows))
display(missing_rows.head())

In [ ]:
# Identifier les indices des lignes où il y a des valeurs manquantes
missing_rows = df[df['boundary_layer_height'].isna() | df['total_column_integrated_water_vapour'].isna()]

print(f"➡️ Nombre total de lignes avec des valeurs manquantes : {len(missing_rows)}")
print("\nExemples de lignes concernées :")
display(missing_rows.head())

# Vérifions la période concernée (utile car les données sont temporelles)
print("\n=== Périodes où les valeurs manquent ===")
missing_periods = missing_rows[['time']].sort_values(by='time')
display(missing_periods.head(10))
display(missing_periods.tail(10))

# vérifier si les valeurs manquantes sont regroupées dans le temps :

In [ ]:
missing_times = df.loc[
    df['boundary_layer_height'].isna() | df['total_column_integrated_water_vapour'].isna(),
    'time'
]
print("Période minimale :", missing_times.min())
print("Période maximale :", missing_times.max())

# Pour voir si elles sont continues ou dispersées
missing_times = pd.to_datetime(missing_times)
missing_gaps = missing_times.diff().dt.total_seconds().dropna()

print("\nDistribution des écarts temporels entre valeurs manquantes :")
display(missing_gaps.describe())


--> mean=min=max=3600s=1h donc toutes les différences entre deux valeurs manquantes consécutives sont exactement 1 heure

--> Vérification manuelle : Janvier : 31 jours → 31 × 24 = 744 h / Février 2024 (année bissextile) : 29 × 24 = 696 h / Mars : 31 × 24 = 744 h / Avril : 30 × 24 = 720 h / Mai : 31 × 24 = 744 h / Juin : 30 × 24 = 720 h et on commence le 1er janvier à 01:00 donc 743 + 696 + 744 + 720 + 744 + 720 = 4367 gaps

--> Les écarts temporels entre les lignes manquantes sont exactement d’une heure, ce qui signifie que les données manquantes suivent la même fréquence horaire que le dataset 

--> Ces valeurs manquantes sont donc réparties régulièrement dans le temps mais représentent une faible proportion des observations (≈1.93%)

--> Pour conserver la cohérence temporelle et ne pas supprimer de données, il est préférable d’appliquer une interpolation temporelle. Cela permettra au modèle de prédiction de garder un signal complet et régulier sans introduire de biais

--> Par la suite, si ces colonnes ne s’avèrent pas pertinentes pour la modélisation, elles pourront être supprimées ou ignorées.

In [ ]:
# === 6. Statistiques descriptives ===
print("\n=== Statistiques descriptives ===")
display(df.describe(include='all').T)

# === 7. Interpolation temporelle ===

In [ ]:
# Conversion de la colonne 'time' en datetime
df['time'] = pd.to_datetime(df['time'])

# Mise en index temporel pour faciliter l'interpolation
df = df.set_index('time')

# Vérification du nombre de valeurs manquantes avant interpolation
print("Valeurs manquantes avant interpolation :")
print(df[['boundary_layer_height', 'total_column_integrated_water_vapour']].isna().sum())

# Interpolation temporelle pour remplir les valeurs manquantes
df['boundary_layer_height'] = df['boundary_layer_height'].interpolate(method='time')
df['total_column_integrated_water_vapour'] = df['total_column_integrated_water_vapour'].interpolate(method='time')

# Vérification après interpolation
print("\n Valeurs manquantes après interpolation :")
print(df[['boundary_layer_height', 'total_column_integrated_water_vapour']].isna().sum())


II - Exploration des variables : 

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline
sns.set(style="whitegrid")
plt.rcParams['figure.figsize'] = (10,5)
from scipy.stats import skew, kurtosis

In [ ]:
print("Nombre de lignes :", df.shape[0])
print("Nombre de colonnes :", df.shape[1])
display(df.head())

 === 1. Statistiques descriptives générales ===

In [ ]:
display(df.describe().T)

# Vérifier les colonnes avec peu de variance ou valeurs uniques
low_var_cols = [c for c in df.columns if df[c].nunique() <= 1]
print("Colonnes avec peu de variance :", low_var_cols)


--> Temperature : moyenne ≈ 12°C, min -10.5°C, max 40.9°C → bonne plage de températures
--> precipitation et rain ont une majorité de 0 → rare mais avec quelques fortes valeurs (max 15.4 mm) 
--> snowfall et snow_depth sont très faibles → peu d’occurrences de neige
--> Vent : wind_speed_10m et wind_speed_100m ont des moyennes modérées mais des valeurs max importantes (52 et 79 m/s)
--> Couverture nuageuse : cloud_cover, cloud_cover_low/mid/high montrent une grande variabilité (écarts-types élevés)
--> Sol : températures et humidité du sol sont stables avec une légère diminution de la variabilité en profondeur
--> Paramètres extrêmes : boundary_layer_height et sunshine_duration ont des valeurs très dispersées (écarts-types très grands)
--> boundary_layer_height et total_column_integrated_water_vapour semblent avoir des valeurs raisonnables et sans NaN (après interpolation)
--> Toutes les colonnes ont beaucoup de valeurs différentes → aucune colonne avec peu de variance (low_var_cols = []). Ça veut dire que toutes les colonnes apportent un peu d’information, on ne va pas supprimer de colonnes juste pour manque de variance

Identifier les types de variables : 

In [ ]:
df.dtypes

Pour les variables catégorielles :

In [ ]:
cat_cols = [c for c in df.columns if df[c].dtype=='int64' and df[c].nunique() < 50 or df[c].dtype=='object']
for c in cat_cols:
    print(f"\nColonne : {c} --> {df[c].nunique()} valeurs uniques")
    print(df[c].value_counts().head(10))


Pour les variables numériques continues : 

In [ ]:
num_cols = [c for c in df.select_dtypes(include=['float64', 'int64']).columns 
            if c not in cat_cols + ['temperature_2m']]  # exclure cat_cols et la cible

print(f"Nombre de variables numériques continues : {len(num_cols)}")
print("Variables :", num_cols)

=== Statistiques descriptives détaillées ===

In [ ]:
descriptives = pd.DataFrame(index=num_cols)

descriptives['count'] = df[num_cols].count()
descriptives['mean'] = df[num_cols].mean()
descriptives['std'] = df[num_cols].std()
descriptives['min'] = df[num_cols].min()
descriptives['25%'] = df[num_cols].quantile(0.25)
descriptives['50%'] = df[num_cols].median()
descriptives['75%'] = df[num_cols].quantile(0.75)
descriptives['max'] = df[num_cols].max()
descriptives['skew'] = df[num_cols].apply(lambda x: skew(x))
descriptives['kurtosis'] = df[num_cols].apply(lambda x: kurtosis(x))
descriptives['zeros (%)'] = df[num_cols].apply(lambda x: (x==0).sum()/len(x)*100)

display(descriptives)

# ===  Distribution des variables ===
for c in num_cols:
    plt.figure(figsize=(8,4))
    sns.histplot(df[c], bins=50, kde=True)
    plt.title(f"Distribution de {c}")
    plt.xlabel(c)
    plt.ylabel("Fréquence")
    plt.show()

--> Humidité relative (relative_humidity_2m) : Moyenne : 76.6 %, assez élevée → il fait globalement humide / Skew négatif (-0.75) → la distribution est légèrement concentrée vers les valeurs élevées (80-100 %).
--> Températures (apparent_temperature, dew_point_2m) : Apparent temperature moyenne : ~10°C, avec des valeurs allant jusqu’à 41.7°C (valeurs extrêmes possibles en été ou journée chaude) / Skew légèrement positif pour apparent temp → quelques valeurs très chaudes.
--> Précipitations, pluie, neige : Très nombreuses valeurs nulles : 85 % pour la pluie, 99 % pour la neige / Distribution très asymétrique (skew très élevé) → la plupart des jours sont secs / Cela signifie que ces variables sont très sparsely populated, et qu’une transformation ou agrégation pourrait être nécessaire.
--> 

Distributions particulières : 
- relative_humidity_2m : Moyenne : 76.6 %, assez élevée → il fait globalement humide / Fortement Asymétrique (vers la gauche), biaisée vers les valeurs élevées --> Standardisation/Normalisation ou une transformation légère (comme une mise à l'échelle puissance) pourrait aider à gérer l'asymétrie
- wind_speed_10m : Très fortement asymétrique vers la droite --> Transformation logarithmique (log) ou racine carrée pour réduire l'asymétrie et traiter les valeurs extrêmes. Ensuite, Standardisation.
- wind_speed_100m : Similaire à wind_speed_10m, mais le mode est plus élevé --> Transformation logarithmique (log) ou racine carrée pour réduire l'asymétrie. Ensuite, Standardisation.
- rain et precipitation : Ces distributions sont dominées par un pic massif à zéro --> Créer une variable binaire "Pluie/Non-Pluie" ou "Précipitations/Non-Précipitations" OU Transformation : Appliquer un log(x+ϵ) sur les valeurs non nulles pour réduire l'asymétrie des rares événements OU Mise à l'échelle : Normalisation ou mise à l'échelle des valeurs.
- snowfall et snow_depth : Extrêmement Asymétrique (Pics à zéro) avec presque toutes les valeurs à zéro --> Variable binaire : Créer une variable binaire "Neige/Non-Neige" pour snowfall et/ou snow_depth > 0. OU Simplification : Étant donné la rareté des valeurs non nulles, ces variables pourraient être moins informatives pour la prédiction de la température future (sauf si la neige est un prédicteur de températures basses)
- wind_speed_10m et wind_speed_100m : Fortement Asymétrique (vers la droite) --> Transformation Logarithmique (log(x+1)) ou Racine Carrée pour réduire l'asymétrie et la variance des valeurs extrêmes. Puis, Standardisation.
- wind_gusts_10m : Fortement Asymétrique (vers la droite), très similaire aux vitesses de vent moyennes, mais avec des valeurs maximales plus élevées --> Transformation Logarithmique (log(x+1)) pour gérer la forte asymétrie et les valeurs extrêmes. Puis, Standardisation.
- wind_direction_10m et wind_direction_100m : Distribution bimodale claire, avec des pics autour de 30-40° (Nord-Est ou Nord) et 220-240° (Sud-Ouest) --> Encodage Cyclique (Sinus/Cosinus). Utiliser la variable brute comme numérique est incorrect (359° est très proche de 0°, mais leur différence numérique est 359), Transformer en sin(θ) et cos(θ) 
- vapour_pressure_deficit : Pic massif près de zéro, avec une longue traîne --> Transformation Logarithmique (log(x+ϵ)) ou Racine Carrée pour gérer l'asymétrie et les zéros. Puis, Standardisation.
- et0_fao_evapotranspiration : Pic massif à zéro ou très proche, avec une longue traîne --> Créer une variable binaire "ET > 0"
- sunshine_duration : Bimodale Extrême / Pics aux extrémités --> Variable Binaire 
- cloud_cover, cloud_cover_low, mid et high : Bimodales Extrêmes --> Normalisation Min-Max (si ce n'est pas déjà fait : 0−100%). Pour certains modèles, une transformation non linéaire peut aider, mais les arbres de décision/boosting gèrent bien cette forme. Une stratégie de segmentation (binaire/catégorielle) pourrait être envisagée (ex: Dégagé <10%, Partiel 10%−90%, Couvert >90%).
- vapour_pressure_deficit : Pic massif près de zéro, longue traîne --> Transformation Logarithmique (log(x+ϵ)) et Standardisation. 
- total_column_integrated_water_vapour : Asymétrique (vers la droite) --> Standardisation/Normalisation. Une légère transformation (log ou racine carrée) pourrait être envisagée pour adoucir la queue, mais la standardisation pourrait être suffisante.
- boundary_layer_height : Fortement Asymétrique (vers la droite) --> Transformation Logarithmique (log(x+1)) pour gérer la forte asymétrie et la longue traîne, puis Standardisation !! 
- soil_moisture_0_to_7cm : Distribution fortement biaisée vers les valeurs élevées --> Standardisation/Normalisation. Aucune transformation Logarithmique n'est nécessaire car elle est biaisée vers la gauche et non vers la droite
- soil_moisture_7_to_28cm, soil_moisture_28_to_100cm et soil_moisture_100_to_255cm : Les distributions sont moins biaisées que la couche superficielle et montrent plusieurs pics. L'humidité du sol en profondeur est plus stable (moins influencée par les événements immédiats), représentant l'état hydrique à long terme. --> Une simple mise à l'échelle est la meilleure approche, en laissant le modèle gérer la multimodalité.

In [ ]:
# === 4. Boxplots pour détecter les valeurs aberrantes ===
for c in num_cols:
    plt.figure(figsize=(8,4))
    sns.boxplot(x=df[c])
    plt.title(f"Boxplot de {c}")
    plt.show()

# === 5. Corrélation avec la cible temperature_2m ===
corr_matrix = df[num_cols + ['temperature_2m']].corr()
corr_with_target = corr_matrix['temperature_2m'].sort_values(ascending=False)
print("\n=== Corrélation des variables numériques continues avec temperature_2m ===")
display(corr_with_target)

# Visualisation des corrélations via heatmap
plt.figure(figsize=(12,10))
sns.heatmap(corr_matrix, annot=False, cmap='coolwarm', center=0)
plt.title("Matrice de corrélation")
plt.show()